In [23]:
!pip install git+https://github.com/pytube/pytube

  Cloning https://github.com/pytube/pytube to /private/var/folders/6d/2nx4lnh53q93x4dnnqysssnh0000gn/T/pip-req-build-ezgq2rqt
  Running command git clone --filter=blob:none --quiet https://github.com/pytube/pytube /private/var/folders/6d/2nx4lnh53q93x4dnnqysssnh0000gn/T/pip-req-build-ezgq2rqt
  Resolved https://github.com/pytube/pytube to commit a32fff39058a6f7e5e59ecd06a7467b71197ce35
  Preparing metadata (setup.py) ... done


In [27]:
!pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.5/758.5 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
import re
import shutil
import subprocess
from pytubefix import Playlist, YouTube

def sanitize_filename(name):
    """Remove or replace unsafe characters from filenames."""
    return re.sub(r'[\\/*?:"<>|]', "_", name).replace(" ", "")

def run(pl):
    # Input destination folder
    filepath = input("Downloads destination (optional): ").strip()
    if filepath == "":
        filepath = os.path.expanduser("~/Downloads/music")
    else:
        filepath = os.path.expanduser(filepath)

    # Create output directory if it doesn't exist
    os.makedirs(filepath, exist_ok=True)

    # Get video links from playlist
    links = pl.video_urls

    for link in links:
        try:
            os.system("clear" if os.name == "posix" else "cls")

            yt = YouTube(link)
            music = yt.streams.filter(file_extension="mp4").first()

            if music is None:
                print(f"No mp4 stream found for: {yt.title}")
                continue

            raw_filename = music.default_filename
            safe_filename = sanitize_filename(raw_filename)
            mp3_filename = safe_filename.replace(".mp4", ".mp3")

            print(f"Downloading {yt.title}...")

            # Download with safe filename
            music.download(filename=safe_filename)

            print("Converting to mp3...")

            # Convert to mp3 using ffmpeg
            try:
                subprocess.run(
                    ["ffmpeg", "-i", safe_filename, mp3_filename],
                    check=True,
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.DEVNULL,
                )
            except subprocess.CalledProcessError:
                print(f"ffmpeg conversion failed for {raw_filename}")
                os.remove(safe_filename)
                continue

            # Move mp3 to destination
            shutil.move(mp3_filename, os.path.join(filepath, mp3_filename))

            # Remove original mp4
            os.remove(safe_filename)

            print(f"Done: {yt.title}\n")

        except Exception as e:
            print(f"Failed to load video: {link}\nError: {e}")

    print("Download finished.")

if __name__ == "__main__":
    url = input("Enter the playlist URL: ").strip()
    pl = Playlist(url)
    run(pl)
